In [1]:
import pandas as pd
import numpy as np

# getting datasets

In [2]:
path = f'../datasets/chicago_trimmed.csv'
df_c = pd.read_csv(path)

In [3]:
path = f'../datasets/law_enforcement_stats.xlsx'
df_l = pd.read_excel(path)

In [4]:
df_c.head()

,date,primary_type,description,location_description,arrest,latitude,longitude
0,2010-10-03 11:30:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926
1,2005-10-31 03:55:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.751061,-87.569876
2,2003-10-07 02:30:00+00:00,BATTERY,SIMPLE,APARTMENT,True,41.751215,-87.571098
3,2009-01-22 02:30:00+00:00,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,41.733650,-87.557845
4,2020-07-03 12:15:00+00:00,BATTERY,SIMPLE,RESIDENCE,False,41.749243,-87.549214


## Taking year from date and adding it to new feature - date_year

In [5]:
df_c['date'] = pd.to_datetime(df_c['date'], errors='coerce')

df_c['Year'] = df_c['date'].dt.year

In [6]:
df_c.head()

,date,primary_type,description,location_description,arrest,latitude,longitude,Year
0,2010-10-03 11:30:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926,2010
1,2005-10-31 03:55:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.751061,-87.569876,2005
2,2003-10-07 02:30:00+00:00,BATTERY,SIMPLE,APARTMENT,True,41.751215,-87.571098,2003
3,2009-01-22 02:30:00+00:00,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",False,41.733650,-87.557845,2009
4,2020-07-03 12:15:00+00:00,BATTERY,SIMPLE,RESIDENCE,False,41.749243,-87.549214,2020


## finding mean and adding it to the missing rows

In [7]:
df_l

,Year,Population,Total Law Enforcement,Officers,Civilians
0,2001,2895995,15066.0,13581.0,1485.0
1,2002,2881295,14932.0,13609.0,1323.0
2,2003,2866361,14777.0,13553.0,1224.0
3,2004,2848996,14325.0,13326.0,999.0
4,2005,2824584,14442.0,13267.0,1175.0
5,2006,2806391,14692.0,13624.0,1068.0
6,2007,2824434,14736.0,13671.0,1065.0
7,2008,2829304,14307.0,13359.0,948.0
8,2009,2848431,13960.0,13088.0,872.0
9,2010,2833649,13318.0,12515.0,803.0


To find the average for 2013 i'm taking the mean of 2012 and 2014 instead of the overall mean. This is to get a more precise number of the specific timeframe. 

In [8]:
officerMean2013 = (df_l['Officers'].iloc[11] + df_l['Officers'].iloc[13]) / 2
civilanMean2013 = (df_l['Civilians'].iloc[11] + df_l['Civilians'].iloc[13]) / 2

print(officerMean2013, civilanMean2013)

11989.0 845.0


In [9]:
officerMean2015 = (df_l['Officers'].iloc[13] + df_l['Officers'].iloc[15]) / 2
civilanMean2015 = (df_l['Civilians'].iloc[13] + df_l['Civilians'].iloc[15]) / 2

print(officerMean2015, civilanMean2015)

11994.0 1024.5


In [10]:
df_l.loc[[12], 'Officers'] = officerMean2013
df_l.loc[[12], 'Civilians'] = civilanMean2013
df_l.loc[[14], 'Officers'] = officerMean2015
df_l.loc[[14], 'Civilians'] = civilanMean2015

In [11]:
df_l

,Year,Population,Total Law Enforcement,Officers,Civilians
0,2001,2895995,15066.0,13581.0,1485.0
1,2002,2881295,14932.0,13609.0,1323.0
2,2003,2866361,14777.0,13553.0,1224.0
3,2004,2848996,14325.0,13326.0,999.0
4,2005,2824584,14442.0,13267.0,1175.0
5,2006,2806391,14692.0,13624.0,1068.0
6,2007,2824434,14736.0,13671.0,1065.0
7,2008,2829304,14307.0,13359.0,948.0
8,2009,2848431,13960.0,13088.0,872.0
9,2010,2833649,13318.0,12515.0,803.0


In [12]:
df_l_trim = df_l.drop(['Total Law Enforcement'], axis=1)

## merging the 2 datasets. adding the civilians, officers and population features to the corresponding year in the chicago dataset 

In [13]:
df_merged = pd.merge(df_c, df_l_trim, on='Year')

In [14]:
df_merged.head()

,date,primary_type,description,location_description,arrest,latitude,longitude,Year,Population,Officers,Civilians
0,2010-10-03 11:30:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926,2010,2833649,12515.0,803.0
1,2010-08-02 04:30:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,True,41.751936,-87.550992,2010,2833649,12515.0,803.0
2,2010-09-17 10:10:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,41.731268,-87.542923,2010,2833649,12515.0,803.0
3,2010-12-31 04:50:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,41.740451,-87.544181,2010,2833649,12515.0,803.0
4,2010-09-08 06:55:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,PARKING LOT/GARAGE(NON.RESID.),False,41.736469,-87.545820,2010,2833649,12515.0,803.0


## renaming featues to standardize format

In [15]:
df_merged = df_merged.rename(columns={"date":"date"})
df_merged = df_merged.rename(columns={"primary_type":"primary_type"})
df_merged = df_merged.rename(columns={"description":"description"})
df_merged = df_merged.rename(columns={"location_description":"location_description"})
df_merged = df_merged.rename(columns={"arrest":"arrest"})
df_merged = df_merged.rename(columns={"latitude":"latitude"})
df_merged = df_merged.rename(columns={"longitude":"longitude"})

df_merged = df_merged.rename(columns={"Year":"year"})
df_merged = df_merged.rename(columns={"Population":"population"})
df_merged = df_merged.rename(columns={"Officers":"officers"})
df_merged = df_merged.rename(columns={"Civilians":"civilians"})

In [16]:
df_merged.head()

,date,primary_type,description,location_description,arrest,latitude,longitude,year,population,officers,civilians
0,2010-10-03 11:30:00+00:00,BATTERY,SIMPLE,SIDEWALK,False,41.745809,-87.547926,2010,2833649,12515.0,803.0
1,2010-08-02 04:30:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,True,41.751936,-87.550992,2010,2833649,12515.0,803.0
2,2010-09-17 10:10:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,41.731268,-87.542923,2010,2833649,12515.0,803.0
3,2010-12-31 04:50:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,41.740451,-87.544181,2010,2833649,12515.0,803.0
4,2010-09-08 06:55:00+00:00,BATTERY,DOMESTIC BATTERY SIMPLE,PARKING LOT/GARAGE(NON.RESID.),False,41.736469,-87.545820,2010,2833649,12515.0,803.0


## save to csv

In [17]:
#df_merged.to_csv('chicago_trimmed_with_police_data.csv', index=False)